# stats: full rec, ransac fit, 2 - 24 Hz

This notebook covers the statistical analysis for spectral slopes that were fit between 2 - 24 Hz using RANSAC regression and the full recordings, which contain both eyes open and eyes closed segments.

In [2]:
%matplotlib inline
import os
import seaborn
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as mpl
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn import linear_model
mpl.rcParams['figure.figsize'] = (16, 10)

## 1. load subject psds and fit slopes

In [8]:
os.chdir('/Users/jorge/Dropbox/research/_psd-slope/src/')
!python /Users/jorge/Dropbox/research/_psd-slope/src/pipeline-full-2s-ransac-oa-ya-02-24.py

/Users/jorge/anaconda/lib/python3.5/site-packages/scipy/linalg/basic.py:884: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)
Saving fitted slopes at:
/Users/jorge/Dropbox/research/_psd-slope/data/pipeline-full/ya-oa-full-ransac-2-24-eyesc-eyeso-sep.csv


## 2. load resulting matrix

In [9]:
df = pd.read_csv('../data/pipeline-full/ya-oa-full-ransac-2-24-eyesc-eyeso-sep.csv')

In [31]:
df.head()

,SUBJECT,CLASS,AGE,SEX,AVG_OA_PSD_EYESC,AVG_OA_PSD_EYESO,AVG_YA_PSD_EYESC,AVG_YA_PSD_EYESO,AVG_PSD_EYESC,AVG_PSD_EYESO,...,B26_EYESO,B27_EYESO,B28_EYESO,B29_EYESO,B30_EYESO,FRONTAL_EYESO,LTEMPORAL_EYESO,CENTRAL_EYESO,RTEMPORAL_EYESO,OCCIPITAL_EYESO
0,120127101,SA_Control,80.84873,2.0,-3.212677,-2.264243,-5.096711,-4.183642,-5.852525,-5.311616,...,-6.837667,-8.136131,-8.812830,-7.736430,-5.903613,-7.499692,-9.870920,-9.256354,-8.411475,-7.131687
1,120127102,MCI_Control,71.89870,1.0,-3.212677,-2.264243,-5.096711,-4.183642,-4.282704,-1.230169,...,-0.520634,-4.107982,-6.186407,-2.693812,-0.639713,-1.417596,-1.803991,-5.032359,-1.305830,-1.603454
2,120127103,MCI_Control,69.43463,2.0,-3.212677,-2.264243,-5.096711,-4.183642,-4.003986,1.255340,...,-1.753993,-5.738814,-6.115665,-6.379150,-3.114273,-4.229089,-5.551192,-6.196523,-4.862521,-2.743728
3,120127104,SA_Control,83.61396,2.0,-3.212677,-2.264243,-5.096711,-4.183642,-2.988003,-1.455893,...,-6.331208,-5.741349,-5.225893,-4.778359,-5.822948,-4.229259,-5.323222,-4.222231,-4.593246,-4.574440
4,120127105,SA_Control,80.69541,2.0,-3.212677,-2.264243,-5.096711,-4.183642,-4.095200,-4.138652,...,-7.897678,-3.850005,-6.484732,-4.840429,-8.813164,-5.924871,-5.432904,-5.899741,-5.781139,-8.530632


## 3. compute t-tests using voytek groups

In [15]:
oa = df[df.AGE >= 60]
oa = oa[oa.CLASS != 'SA']
ya = df[df.AGE <= 35.5]
channels = list(df.columns.values)[4:] # Exclude SUBJECT, CLASS, AGE, SEX

In [29]:
for ch in channels:
    result = stats.ttest_ind(ya[ch], oa[ch], equal_var=False)
    if result[1] < 0.05:
                print("{}:\t\t\t{:.2f},\t{:.3f}".format(ch, result.statistic, result.pvalue))

AVG_OA_PSD_EYESC:			-5.48,	0.000
AVG_OA_PSD_EYESO:			-2.49,	0.017
AVG_YA_PSD_EYESO:			6.00,	0.000


## 4. construct plots comparing groups

In [25]:
result = stats.ttest_ind(ya['AVG_OA_PSD_EYESC'], oa['AVG_OA_PSD_EYESC'], equal_var=True)
print(result)

Ttest_indResult(statistic=-5.9926425478149747, pvalue=9.5391665484706531e-08)


In [28]:
result.statistic

-5.9926425478149747

In [27]:
oa['AVG_OA_PSD_EYESC']

0    -3.212677
1    -3.212677
2    -3.212677
3    -3.212677
4    -3.212677
5    -3.212677
7    -3.212677
8    -3.212677
9    -3.212677
10   -3.212677
11   -3.212677
13   -3.212677
15   -3.212677
16   -3.212677
20   -3.212677
22   -3.212677
23   -3.212677
24   -3.212677
25   -3.212677
26   -3.212677
27   -3.212677
32   -3.212677
35   -3.212677
38   -3.212677
39   -3.212677
40   -3.212677
42   -3.212677
44   -3.212677
45   -3.212677
46   -3.212677
47   -3.212677
48   -3.212677
49   -3.212677
50   -3.212677
51   -3.212677
52   -3.212677
53   -3.212677
Name: AVG_OA_PSD_EYESC, dtype: float64

In [3]:
df = pd.read_csv('../data/pipeline-full/ya-oa-full-ransac-2-24-eyesc-eyeso-sep.csv')

OSError: File b'../data/pipeline-full/ya-oa-full-ransac-2-24-eyesc-eyeso-sep.csv' does not exist